In [4]:
import difflib
import re

In [ ]:
import sys
print(sys.executable)

## Comparison of strings 
This tool can be used for comparing the extracted odes for the same model, but with different prompts or architectures.

> s1 is the original string, s2 is the extracted

In [5]:
version = "001"

data_path = '.'  # Current directory, or change to your path
df_correct = pd.read_csv(os.path.join(data_path, 'correct_eqs_list.tsv'), sep='\t')
df_extracted = pd.read_csv(os.path.join(data_path, f'extracted_eqs_VERSION{version}.tsv'), sep='\t')

NameError: name 'pd' is not defined

In [ ]:
output_file= f'comparison_results_version{version}.txt'

with open(output_file, 'w') as f:
        for idx, row in df_correct.iterrows():
            model_name = row['model']
            s1 = row['correct_eqs']

            matching = df_extracted[df_extracted['model'] == model_name]
            if not matching.empty:
                s2 = matching.iloc[0]['extracted_eqs'] #there shouldn't be multiple rows for a model, but iloc[0] selects the first one if it happens

                f.write(f"\n{'='*80}\n")
                f.write(f"Model: {model_name}\n")
                f.write(f"{'='*80}\n")
                f.write("Correct equation:\n")
                f.write(s1 + "\n\n")
                f.write("Extracted equation:\n")
                f.write(s2 + "\n\n")
                f.write('Above-below comparison\n')
                f.write('-'*80 + '\n')

                eq1, eq2 = equalize(s1, s2)
                f.write(eq1 + '\n')
                f.write(eq2 + '\n')
                f.write('\n')

                print(f"\nComparing model: {model_name}")
                print('Above-below comparison')
                print('-'*80)
                show_comparison(s1, s2, sidebyside=False)

print(f"\nResults saved to: {output_file}")

In [ ]:
def tokenize(s):
    return re.split('\s+', s)
def untokenize(ts):
    return ' '.join(ts)
        
def equalize(s1, s2):
    l1 = tokenize(s1)
    l2 = tokenize(s2)
    res1 = []
    res2 = []
    prev = difflib.Match(0,0,0)
    for match in difflib.SequenceMatcher(a=l1, b=l2).get_matching_blocks():
        if (prev.a + prev.size != match.a):
            for i in range(prev.a + prev.size, match.a):
                res2 += ['_' * len(l1[i])]
            res1 += l1[prev.a + prev.size:match.a]
        if (prev.b + prev.size != match.b):
            for i in range(prev.b + prev.size, match.b):
                res1 += ['_' * len(l2[i])]
            res2 += l2[prev.b + prev.size:match.b]
        res1 += l1[match.a:match.a+match.size]
        res2 += l2[match.b:match.b+match.size]
        prev = match
    return untokenize(res1), untokenize(res2)

def insert_newlines(string, every=64, window=10):
    result = []
    from_string = string
    while len(from_string) > 0:
        cut_off = every
        if len(from_string) > every:
            while (from_string[cut_off-1] != ' ') and (cut_off > (every-window)):
                cut_off -= 1
        else:
            cut_off = len(from_string)
        part = from_string[:cut_off]
        result += [part]
        from_string = from_string[cut_off:]
    return result

def show_comparison(s1, s2, width=40, margin=10, sidebyside=True, compact=False):
    s1, s2 = equalize(s1,s2)

    if sidebyside:
        s1 = insert_newlines(s1, width, margin)
        s2 = insert_newlines(s2, width, margin)
        if compact:
            for i in range(0, len(s1)):
                lft = re.sub(' +', ' ', s1[i].replace('_', '')).ljust(width)
                rgt = re.sub(' +', ' ', s2[i].replace('_', '')).ljust(width) 
                print(lft + ' | ' + rgt + ' | ')        
        else:
            for i in range(0, len(s1)):
                lft = s1[i].ljust(width)
                rgt = s2[i].ljust(width)
                print(lft + ' | ' + rgt + ' | ')
    else:
        print(s1)
        print(s2)

Run as shown in the example above, use of multiple quotation marks is needed.

In [3]:
original_str = """odes = [
    sympy.Eq(S(t).diff(t), - beta * S(t) * I(t) / N),
    sympy.Eq(E(t).diff(t), beta * S(t) * I(t) / N - alpha * E(t)),
    sympy.Eq(I(t).diff(t), alpha * E(t) - gamma * I(t)),
    sympy.Eq(R(t).diff(t), gamma * I(t))
]"""

extracted_str = """odes = [
    sympy.Eq(S(t).diff(t), - beta * S(t) / N),
    sympy.Eq(I(t).diff(t), beta * S(t) * I(t) / N - alpha * E(t)),
    sympy.Eq(I(t).diff(t), alpha * E(t) - gamma_2 * I(t)),
    sympy.Eq(R(t).diff(t), gamma * I(t))
]"""

print(original_str)
print(extracted_str)

print()
print('Above-below comparison')
print('-------------------------------------------------------------------------------------')
show_comparison(original_str, extracted_str, sidebyside=False)

odes = [
    sympy.Eq(S(t).diff(t), - beta * S(t) * I(t) / N),
    sympy.Eq(E(t).diff(t), beta * S(t) * I(t) / N - alpha * E(t)),
    sympy.Eq(I(t).diff(t), alpha * E(t) - gamma * I(t)),
    sympy.Eq(R(t).diff(t), gamma * I(t))
]
odes = [
    sympy.Eq(S(t).diff(t), - beta * S(t) / N),
    sympy.Eq(I(t).diff(t), beta * S(t) * I(t) / N - alpha * E(t)),
    sympy.Eq(I(t).diff(t), alpha * E(t) - gamma_2 * I(t)),
    sympy.Eq(R(t).diff(t), gamma * I(t))
]

Above-below comparison
-------------------------------------------------------------------------------------
odes = [ sympy.Eq(S(t).diff(t), - beta * S(t) * I(t) / N), sympy.Eq(E(t).diff(t), ______________________ beta * S(t) * I(t) / N - alpha * E(t)), sympy.Eq(I(t).diff(t), alpha * E(t) - gamma _______ * I(t)), sympy.Eq(R(t).diff(t), gamma * I(t)) ]
odes = [ sympy.Eq(S(t).diff(t), - beta * S(t) _ ____ / N), ______________________ sympy.Eq(I(t).diff(t), beta * S(t) * I(t) / N - alpha * E(t)), sympy.Eq(I(t).diff(t), alpha * E(t) - _____ g